In [1]:
import numpy as np
from scipy.special import softmax
from scipy.io import mmread
from scipy.stats import entropy
import time

In [5]:
A = np.eye(3)
A

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [11]:
from scipy.sparse.linalg import eigs
p_stationary = np.real(eigs(A.T, k=1, sigma=1.0000001)[1])

In [12]:
p_stationary.T @ A - p_stationary.T

array([[0., 0., 0.]])

# Objective function
Sample $(x_i, y_i)\sim W$ from a joint distribution $W\in\mathbb{R}^{N\times N}$. For a matrix $L\in\mathbb{R}^{N\times N}$, we take the softmax $\sigma$ _row-wise_ to make it a conditional distribution, and look at the maximum likelihood objective

$$
\text{max}_L \prod_{x_i, y_i}\sigma(L)_{y_i | x_i}\,.
$$

Taking $-\text{log}$, the objective reformulates to the loss

\begin{aligned}
&\text{min}_L -\sum_{x_i, y_i}\log\left(\sigma(L)_{y_i | x_i}\right)\\
\Leftrightarrow\quad &\text{min}_L  -\sum_{i, j}\frac{\#\{(x, y) = (i, j)\}}{\# \text{samples}}\log\sigma(L)_{j | i}\\
\Leftrightarrow\quad &\text{min}_L  -\sum_{i, j}w_{i,j}\log\sigma(L)_{j | i}\\
\Leftrightarrow\quad &\text{min}_L  -\mathbb{E}_{W(x, y)}\left[\log\sigma(L)(y | x)\right]\\
\Leftrightarrow\quad &\text{min}_L -\mathbb{E}_{W(x, y)}\left[\log W(y | x)\right]
+ \mathbb{E}_{W(x, y)}\left[\log\frac{W(y|x)}{\sigma(L)(y | x)}\right]\\
\Leftrightarrow\quad &\text{min}_L\quad H_W(Y | X)
\quad +\quad D\left(W(Y|X) \;||\; \sigma(L)(Y|X)\right)
\end{aligned}

So we need to minimize the loss that consists of conditional entropy $H_W(Y | X)$ for the target distribution, and conditional relative entropy $D\left(W(Y|X) \;||\; \sigma(L)(Y|X)\right)$ between target distribution and learned conditional distribution $\sigma(L)$.

For the gradient $\nabla F$ of the loss $F$, we need to split the joint distribution $W$ into marginal $M$ (diagonal matrix) and conditional distribution $P$, s.t. $W = M \cdot P$. The gradient is then (just plug in softmax and compute $\partial L_{i^\prime, j^\prime}$) given by

$$
\nabla F(L) = M \cdot \left(\sigma(L) - P\right)\,.
$$

In [2]:
def weighted_logreg_grad(L, P, M):
    """
    Computes the gradient of the weighted cross-entropy loss in L with weight matrix M * P.
    Parameters
    ----------
    L: np.array of shape (N, N)
            Logits of learnable (low rank) transition matrix.
    M: diagonal np.array of shape (N, N)
            Marginal of weight distribution.
    P: np.array of shape (N, N)
            Transition matrix of weight distribution.

    Returns
    -------
    grad: np.array of shape (N, N)
            Gradient of loss at L.
    """
    grad = M @ (softmax(L, axis=-1) - P)
    return grad

In [3]:
# def weighted_logreg_loss(L, P, M):
#     """
#     Computes the weighted cross-entropy loss in L with weight matrix M * P.
#     Parameters
#     ----------
#     L: np.array of shape (N, N)
#             Logits of learnable (low rank) transition matrix.
#     M: diagonal np.array of shape (N, N)
#             Marginal of weight distribution.
#     P: np.array of shape (N, N)
#             Transition matrix of weight distribution.

#     Returns
#     -------
#     loss: float
#             Loss at L.
#     """    
#     W = M @ P
#     M_eL = np.diag(np.log(np.sum(np.exp(L), axis=-1)))
#     loss = np.sum(M_eL @ W - L * W)
#     return loss

In [50]:
def weighted_logreg_loss(L, W):
    """
    Computes the weighted cross-entropy loss in L with weight matrix M * P.
    Parameters
    ----------
    L: np.array of shape (N, N)
            Logits of learnable (low rank) transition matrix.
    W: np.array of shape (N, N)
            Matrix of weight distribution.

    Returns
    -------
    loss: float
            Loss at L.
    """    
    d = np.log(np.exp(L).sum(axis=-1, keepdims=True))
    loss = np.sum(W * (d * np.ones_like(W) - L))
    return loss

In [51]:
class LogReg(object):
    def __init__(self, P, M):
        self.P = P
        self.M = M
        self.L = np.zeros_like(P)
        self.step = 0
        
    def __call__(self):
        return self.L, softmax(self.L, axis=-1)
        
    def train(self, steps, lr):
        for self.step in range(self.step, self.step + steps):
            self.L -= lr * weighted_logreg_grad(self.L, self.P, self.M)
            loss = weighted_logreg_loss(self.L, self.P, self.M)
#             if self.step % 50 == 50-1:
#                 print(f'Step: {self.step}, Loss: {loss:.5f}')

In [52]:
def conditional_relative_entropy(W):
    D = np.diag(1 / np.sum(W, axis=-1))
    P = D @ W
    entropy = -np.sum(W[W!=0] * np.log(P[W!=0]))
    return entropy

### Example: Learning the adjacency matrix for Zachary's Karate Club

In [56]:
G = mmread('../data/soc-karate.mtx')
A = G.toarray()

W = A / A.sum()
# d = np.sum(A, axis=-1)
# M = np.diag(d) / d.sum()
# P = (1 / np.expand_dims(d, -1)) * A

# W = M @ P # = A / A.sum()

In [57]:
logreg = LogReg(P=P, M=M)

In [55]:
start_time = time.time()
logreg.train(steps=500, lr=1)
end_time = time.time()

L, P_L = logreg()
print(f"Loss: {weighted_logreg_loss(L=L, P=P, M=M)} in {end_time - start_time} seconds.")

TypeError: weighted_logreg_loss() takes 2 positional arguments but 3 were given

In [39]:
best_loss = conditional_relative_entropy(W)
print('Difference to best loss is {}'.format(weighted_logreg_loss(L=L, P=P, M=M)-best_loss))

Difference to best loss is 0.0779378362012868


In [61]:
conditional_relative_entropy(W)

1.788998746671057

In [62]:
weighted_logreg_loss(L=L, W=W)

1.866936582872344

In [66]:
L_W = -100 * np.ones_like(W)
L_W[W!=0] = np.log(W[W!=0])
print(L_W)

[[-100.           -5.04985601   -5.04985601 ...   -5.04985601
  -100.         -100.        ]
 [  -5.04985601 -100.           -5.04985601 ... -100.
  -100.         -100.        ]
 [  -5.04985601   -5.04985601 -100.         ... -100.
    -5.04985601 -100.        ]
 ...
 [  -5.04985601 -100.         -100.         ... -100.
    -5.04985601   -5.04985601]
 [-100.         -100.           -5.04985601 ...   -5.04985601
  -100.           -5.04985601]
 [-100.         -100.         -100.         ...   -5.04985601
    -5.04985601 -100.        ]]


In [67]:
weighted_logreg_loss(L=L_W, W=W)

1.788998746671057

### Example 2) Joint distributions without zeros

In [12]:
N = 34
alpha = 3

P_trans = np.random.dirichlet(alpha * np.ones(N), size=N)
M=np.eye(N)/N
print(np.min(P_trans))

0.0017578789977912472


In [13]:
logreg = LogReg(P=P_trans, M=M)

In [14]:
logreg.train(steps=5000, lr=0.1)

L, P_L = logreg()

In [15]:
best_loss = conditional_relative_entropy(M @ P_trans)
print(best_loss)

3.3687511020399263


In [16]:
weighted_logreg_loss(L=L, P=P_trans, M=M)

3.4336667704710555

In [17]:
print('Difference to best loss is {}'.format(weighted_logreg_loss(L=L, P=P_trans, M=M)-best_loss))

Difference to best loss is 0.06491566843112917
